# Descriptive Run


## Set Up Spark Session

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()


24/11/26 05:01:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Authenticate arcgis api for python
Use https://ucr.maps.arcgis.com/ to sign in with ucr credentials so you have access to arcgis pro and api development permissions.

Then go to content > new item > developer credential > continue > then type in redirect url = urn:ietf:wg:oauth:2.0:oob and url you can use https://localhost. This will create the client_id, which you should paste into the following cell, and then when you run it it should send you a password link. Type the password link into the box.



In [2]:
from arcgis.gis import GIS
import arcgis
from arcgis.map import Map
print(arcgis.__version__)


# Authenticate GIS.
my_client_id = "Sry1p3Nb1sg3fciJ"
gis = GIS("https://ucr.maps.arcgis.com", client_id=my_client_id)

2.4.0
Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucr.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Sry1p3Nb1sg3fciJ&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=61pZN6vIuglIrVqaCcbzNBNfdlgMuC&allow_verification=false


Enter code obtained on signing in using SAML:  ········


24/11/26 05:01:47 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Access featurecollection of riverside reporting districts

Using map from owner:"KPrakah@riversideca.gov_CityOfRiverside"


In [19]:
# Access a web map of riverside reporting districts (RDs)
item = gis.content.get("4855e57db5d3430cb94ddd32688ab7b7")
feature_collection = item.layers[0]

In [20]:
# Update Esri Requirement
feature_collection.properties.layerDefinition.htmlPopupType = 'esriServerHTMLPopupTypeAsHTMLText'
# Display what a feature looks like. This gets a featureSet from a featureCollection, then displays
# the first feature in the feature set.
feature_collection.query().features[0]

{"geometry": {"rings": [[[-13059322, 4013797], [-13059306, 4013790], [-13059307, 4013320], [-13059241, 4013320], [-13059241, 4013420], [-13059108, 4013418], [-13059090, 4013415], [-13059128, 4013345], [-13059039, 4013291], [-13059101, 4013120], [-13059306, 4013120], [-13059307, 4011761], [-13061269, 4011727], [-13061268, 4013666], [-13061036, 4013698], [-13060068, 4013920], [-13059782, 4013912], [-13059322, 4013797]]], "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 0, "OBJECTID": 1, "NPC": "EAST", "NAME": "K03"}}

## Query Riverside_Crime_reports using Spark by Reporting District

In [70]:

from pyspark.sql.functions import col


df = spark.read.csv("Riverside_Crime_Reports.csv", header=True, inferSchema=True)
df.columns
# Filter rows where crimeType is 'ROBBERY' and group by rpc column, counting the occurrences
result = (df.select("rd", "crimeType")
          .groupBy("rd")
          .count()
          .select("rd", "Count")
          .withColumnRenamed("rd", "NAME"))

# Show the result
result.show()
pd_crime = result.toPandas()

+----+-----+
|NAME|Count|
+----+-----+
| B05|  362|
| Z21|   25|
| E02|  983|
| G12|  367|
| 17A|  288|
| B01|  416|
| I01|  385|
| K03|  624|
| I06|  488|
| D16|  429|
| F01| 1060|
| I04| 1268|
| I14|  162|
| F04|  145|
| B06|  314|
| I03|  850|
|   3|    5|
| H14| 1800|
| A01| 2008|
| I02|  804|
+----+-----+
only showing top 20 rows



## Merge these counts into the FeatureCollection

In [71]:
print("before:")
feature_collection.query().features[0]

before:


{"geometry": {"rings": [[[-13059322, 4013797], [-13059306, 4013790], [-13059307, 4013320], [-13059241, 4013320], [-13059241, 4013420], [-13059108, 4013418], [-13059090, 4013415], [-13059128, 4013345], [-13059039, 4013291], [-13059101, 4013120], [-13059306, 4013120], [-13059307, 4011761], [-13061269, 4011727], [-13061268, 4013666], [-13061036, 4013698], [-13060068, 4013920], [-13059782, 4013912], [-13059322, 4013797]]], "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 0, "OBJECTID": 1, "NPC": "EAST", "NAME": "K03"}}

In [74]:
fset = feature_collection.query().sdf['SHAPE'].head()
fset

0    {"rings": [[[-13059322, 4013797], [-13059306, ...
1    {"rings": [[[-13061268, 4013817], [-13061269, ...
2    {"rings": [[[-13073478, 4013648], [-13074932, ...
3    {"rings": [[[-13076145, 4014008], [-13076824, ...
4    {"rings": [[[-13068909, 4015593], [-13067078, ...
Name: SHAPE, dtype: geometry

In [34]:
import pandas as pd
from arcgis.features import FeatureSet

#convert featureset to dataframe, merge the dataframes, then convert back to a featureset
fset_df = fset.sdf
merged_df = pd.merge(pd_crime, fset_df, on="NAME") 
fset2 = FeatureSet.from_dataframe(merged_df)

# create new featurecollection from the featureset
from arcgis.features import FeatureCollection
feature_collection2 = FeatureCollection.from_featureset(fset2)

In [35]:
print("after:")

feature_collection2.query().features[0]

after:


{"geometry": {"rings": [[[-13058817, 4026512], [-13058816, 4026065], [-13058771, 4026002], [-13058753, 4025929], [-13058765, 4025784], [-13058754, 4025637], [-13058787, 4025537], [-13059786, 4025535], [-13059788, 4026032], [-13060099, 4026103], [-13060299, 4026191], [-13060467, 4026309], [-13060658, 4026518], [-13058817, 4026512]]], "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"NAME": "B05", "Count": 362, "FID": 113, "OBJECTID": 114, "NPC": "EAST"}}

## Create a map to display the data. You will be able to click on each reporting district to see statistics.

In [36]:
# Create the map using our featureCollection
my_map = Map()
my_map.basemap.basemap =  "arcgis-streets"
my_map.content.add(feature_collection2)

In [37]:

# Update the information that will be shown when clicking on a reporting district

from arcgis.map.popups import FieldInfo, PopupExpressionInfo
expression_infos = [
    PopupExpressionInfo(
        title="Name",
        expression="return $feature.NAME;"
    )
]

field_infos = [
    FieldInfo(
        fieldName="NAME",
        label="Reporting District",
        visible=True
    ),
    FieldInfo(
        fieldName="Count",
        label="Number of Police Reports",
        visible=True
    )
]
my_map.content.popup(0).edit(title="HELLO", expression_infos=expression_infos, field_infos=field_infos)

True

In [42]:
import ipywidgets as widgets


layer_selector = widgets.Dropdown(
    options={
        "Layer 1",
        "Layer 2"
    },
    description="Layer:"
)



# Update starting field of view, and display the map
my_map.extent = {'spatialReference': {'wkid': 102100}, 'xmin': -13087454.153546248, 'ymin': 4009445.945260928, 'xmax': -13044725.85473489, 'ymax': 4032377.0537464498}

widgets.VBox([layer_selector, my_map])
# # Display map
# my_map


# Here is the more organized but less descritpive rundown. It is organized as follows:


# Authenticate

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()

24/11/26 06:41:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.



Use https://ucr.maps.arcgis.com/ to sign in with ucr credentials so you have access to arcgis pro and api development permissions.

Then go to content > new item > developer credential > continue > then type in redirect url = urn:ietf:wg:oauth:2.0:oob and url you can use https://localhost. This will create the client_id I believe which is what I used below. Use your own client id in the next cell, and then when you run it it should send you a password link. Type the password link into the box.

In [2]:
from arcgis.gis import GIS
import arcgis
from arcgis.map import Map
print(arcgis.__version__)


# Authenticate GIS.
my_client_id = "Sry1p3Nb1sg3fciJ"
gis = GIS("https://ucr.maps.arcgis.com", client_id=my_client_id)

24/11/26 06:41:56 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


2.4.0
Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucr.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Sry1p3Nb1sg3fciJ&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=tA8c992H8UxIbt5oUNJJYIa0VuLFxO&allow_verification=false


Enter code obtained on signing in using SAML:  ········


# Global Data

In [3]:
    crimeTypes = ['DRUG ABUSE VIOLATIONS', 'OTHER: DUI', 'ASSAULT: Other Assaults',
           'OTHER: All Other Offenses', 'WEAPONS: Carrying, Possessing, etc.',
           'Other Assaults - Not Aggravated',
           'ASSAULT: Knife or cutting instrument', 'THEFT: Shoplifting',
           'THEFT: Theft from motor vehicle', 'ASSAULT: Firearm',
           'SEX CRIMES: Sex Offenses', 'OTHER: Stolen Property',
           'FRAUD: Other Fraud', 'FRAUD: Forgery & Counterfeiting',
           'VANDALISM: Vandalism', 'MOTOR VEH. THEFT: Other vehicles',
           'BURGLARY: Forcible entry', 'DRUG ABUSE VIOLATIONS: Drunkenness',
           'FRAUD: Embezzlement', 'THEFT: All other larceny',
           'MOTOR VEH. THEFT: Autos', 'MOTOR VEH. THEFT: Trucks and buses',
           'ASSAULT: Strong-Arm', 'ASSAULT: Other dangerous weapon',
           'ROBBERY: Firearm', 'ROBBERY: Strong-Arm',
           'THEFT: Theft of motor vehicle parts or accessories',
           'THEFT: Theft of bicycles', 'BURGLARY: Unlawful entry - no force',
           'SEX CRIMES: Sexual Assault', 'OTHER: Offenses Against Family',
           'THEFT: Theft from buildings',
           'THEFT: Theft from coin-operated machine or device',
           'DISTURBING THE PEACE: Disorderly Conduct',
           'THEFT: Pocket picking', 'BURGLARY: Attempted forcible entry',
           'ROBBERY: Knife or cutting instrument',
           'OTHER: Curfew and Loitering',
           'DRUG ABUSE VIOLATIONS: Liquor Laws',
           'HOMICIDE: Murder & non-negligent manslaughter', 'GAMBLING',
           'ROBBERY: Other dangerous weapon', 'SEX CRIMES: Prostitution/Vice',
           'THEFT: Purse snatching', 'DRIVING UNDER THE INFLUENCE',
           'DESTRUCTION / DAMAGE / VANDALISM OF PROPERTY', 'SIMPLE ASSAULT',
           'ALL OTHER OFFENSES', 'WEAPON LAW VIOLATIONS',
           'FAMILY OFFENSES, NONVIOLENT', 'BURGLARY/BREAKING AND ENTERING',
           'AGGRAVATED ASSAULT',
           'THEFT OF MOTOR VEHICLE PARTS OR ACCESSORIES', 'ALL OTHER LARCENY',
           'CREDITCARD / AUTOMATED TELLER MACHINE FRAUD', 'SHOPLIFTING',
           'DISORDERLY CONDUCT', 'MOTOR VEHICLE THEFT',
           'KIDNAPPING / ABDUCTION', 'STOLEN PROPERTY OFFENSES',
           'THEFT FROM BUILDING', 'FALSEPRETENSES / SWINDLE / CONFIDENCEGAME',
           'TRESPASS OF REAL PROPERTY', 'ROBBERY', 'IDENTITYTHEFT',
           'DRUG EQUIPMENT VIOLATIONS', 'IMPERSONATION',
           'DRUG NARCOTIC VIOLATIONS', 'THEFT FROM MOTOR VEHICLE',
           'SEX CRIMES', 'POCKET-PICKING', 'EMBEZZLEMENT',
           'HACKING / COMPUTERINVASION', 'COUNTERFEITING / FORGERY',
           'LIQUOR LAW VIOLATIONS', 'INTIMIDATION',
           'PORNOGRAPHY / OBSCENEMATERIAL',
           'CURFEW / LOITERING / VAGRANCY VIOLATIONS',
           'EXTORTION / BLACKMAIL', 'PROSTITUTION', 'WELFAREFRAUD',
           'MURDER AND NON-NEGLIGENT MANSLAUGHTER', 'ARSON',
           'THEFT FROM COIN-OPERATED MACHINE OR DEVICE', 'ANIMAL CRUELTY',
           'PURSE-SNATCHING', 'HUMANTRAFFICKING ,SERVITUDE INVOLUNTARY',
           'WIREFRAUD', 'OPERATING / PROMOTING / ASSISTING GAMBLING']

# Create Police Report Dataframe

In [4]:
def create_report_dataframe():
    df = spark.read.csv("Riverside_Crime_Reports.csv", header=True, inferSchema=True)
    return df
crime_df = create_report_dataframe()
crime_df.head()

Row(caseNumber=200000046, reportDate=datetime.date(2020, 1, 1), offenseDate=datetime.date(2020, 1, 1), callTime=datetime.datetime(2024, 11, 26, 15, 27, 51), crimeType='DRUG ABUSE VIOLATIONS', premise='HIGHWAYS', blockAddress='Sagittarius Dr / Boundary Ln', rd='J14', npc='WEST', _id=146205)

# Create shape dataframe

In [5]:
# crime_df.toPandas()["blockAddress"].unique()
from arcgis.geometry import Point
from arcgis.geocoding import geocode
# geocode('Sagittarius Dr / Boundary Ln, Riverside, CA')[0]

def geocode_block_address(block_address: str):
    if not block_address or not isinstance(block_address, str):  # Check for None or invalid input
        return None  # Return None for invalid addresses

    try:
        # Perform geocoding
        # results = geocode({"Street": block_address, "City": "Riverside", "State": "CA"})
        # print({"Street": block_address, "City": "Riverside", "State": "CA"})
        results = geocode(block_address, ", Riverside, CA")
        if len(results) == 0:
            return None
        best_result = results[0]
        print(best_result)
        best_result['location'].update({"spatialReference": {"wkid": 4326}})
        return Point(best_result['location'])
    except Exception as e:
        # Handle geocoding errors
        print(f"Error geocoding address '{block_address}': {e}")
        raise RuntimeError("Probably need to restart execution of this cell after re-authenticating arcgis.")
        return None



In [6]:

import pandas as pd
from arcgis.features import FeatureSet
def create_shape_dataframe():
    # Access a web map of riverside reporting districts (RDs)
    item = gis.content.get("4855e57db5d3430cb94ddd32688ab7b7")
    feature_collection = item.layers[0]
    # Update Esri Requirement
    feature_collection.properties.layerDefinition.htmlPopupType = 'esriServerHTMLPopupTypeAsHTMLText'
    
    fset = feature_collection.query()
    fset_df = fset.sdf
    return fset_df[['NAME', 'SHAPE']].rename(columns={"NAME": "rd"})


import pickle


def create_point_dataframe(crime_df, file_path = ''):
    
    crime_df_pd = crime_df.toPandas()
    invalid_points = []
    
    if os.path.exists(file_path):
        with open(file_path, 'rb') as file:
            geocoded_points = pickle.load(file)
        print("Loaded geocoded points from file.")
    else:
        print("Cannot find file with pre-generated geocodings. Geocoding all points. This may take a while and require re-authenticating arcgis. Consider uploading the file containing the correct geocoded locations.")
        geocoded_points = {}
    
    
    # Go through every block address. Remembering invalid points for error checking with response not available
    for i, blk_address in enumerate(crime_df_pd['blockAddress']):
        if i % 500 == 0:
            with open(file_path, 'wb') as file:
                pickle.dump(geocoded_points, file)
    
        if blk_address in geocoded_points:
            continue
        else:
            geocoded_point = geocode_block_address(blk_address)
            geocoded_points[blk_address] = geocoded_point
        if geocoded_point is None:
            invalid_points.append(i)
    
    with open(file_path, 'wb') as file:
        pickle.dump(geocoded_points, file)

    
    blockAddress_geometry = pd.DataFrame({'blockAddress': crime_df.toPandas()['blockAddress'].unique()})
    blockAddress_geometry['SHAPE'] = blockAddress_geometry['blockAddress'].apply(lambda blk_address: geocoded_points[blk_address])
    return blockAddress_geometry

blockAddress_geometry = create_point_dataframe(crime_df, "geocoded_points.pkl")    
rd_geometry = create_shape_dataframe()

Loaded geocoded points from file.


In [7]:
blockAddress_geometry.head()

,blockAddress,SHAPE
0,Sagittarius Dr / Boundary Ln,"{'x': -117.456645469102, 'y': 33.895308997468,..."
1,10300 BLOCK CAMPBELL AVE,"{'x': -117.478330717128, 'y': 33.936321735676,..."
2,5800 BLOCK CHALLEN AVE,"{'x': -117.46579873813, 'y': 33.941310611459, ..."
3,E Alessandro Blvd / Mission Grove Pkwy,"{'x': -117.330934916792, 'y': 33.917409046214,..."
4,2900 BLOCK MARY ST,"{'x': -117.38877287487, 'y': 33.936060077028, ..."


# Query Police Report Dataframe

In [8]:

from pyspark.sql.functions import col

# ex. query_reports(crime_df, "ROBBERY", 1, 120) would query for less than 120 robberies within the area

def query_reports(df, crimeType: str = "", condition_code: int = 0, condition: int = 0, points: bool = False):
    # Filter rows where crimeType is 'ROBBERY' and group by rpc column, counting the occurrences
    query_present = False
    query_string = ''

    # Handle incorrect input for query_string
    if crimeType != "":
        if crimeType in crimeTypes:
            print("Querying... ")
            query_string = crimeType
            query_present = True
        else:
            print("Warning: Invalid Crime Type")
            query_string = 'Invalid Query: Returning All Crimes'
            query_present = False
    else:
        query_string = 'All Crimes'
        
    
    if query_present:
        query_result = (df.select("rd", "blockAddress", "crimeType")
                      .filter(df.crimeType == crimeType))
    else:
        query_result = df

    queried_df_blockAddress = (query_result.select("blockAddress")
                         .groupby("blockAddress")
                         .count())
    
    queried_df_rd = (query_result.select("rd", "crimeType")
                      .groupBy("rd")
                      .count())
    
    print("Querying Finished")
    queried_df_blockAddress = queried_df_blockAddress.toPandas()
    queried_df_rd = queried_df_rd.toPandas()
    
    return queried_df_blockAddress, queried_df_rd, query_string
    
queried_df_blockAddress, queried_df_rd, query_string = query_reports(crime_df)
display(queried_df_blockAddress)
display(queried_df_rd)

Querying Finished


,blockAddress,count
0,3500 BLOCK POLK ST,132
1,1000 BLOCK SPRUCE ST,188
2,5th St / Pine St,1
3,Lila St/jefferson St,1
4,4500 BLOCK ARLINGTON AVE,35
...,...,...
14109,Stover Ave / Gaylord St,1
14110,Manchester Pl / California Ave,1
14111,6400 BLOCK RHONDA RD,1
14112,5200 BLOCK MARENGO CT,1


,rd,count
0,B05,362
1,Z21,25
2,E02,983
3,G12,367
4,17A,288
...,...,...
209,g03,2
210,e15,1
211,H17,1
212,PG1,1


# Merge Dataframes

In [9]:
def merge_dataframes(queried_df_pd, geometry_df_pd, col):
    merged_df = pd.merge(queried_df_pd, geometry_df_pd, on=col, how="outer")
    merged_df["count"] = merged_df["count"].fillna(0).astype(int)
    return merged_df

df = merge_dataframes(queried_df_rd, rd_geometry, "rd")
df2 = merge_dataframes(queried_df_blockAddress, blockAddress_geometry, "blockAddress")
display(df)
display(df2)


,rd,count,SHAPE
0,B05,362,"{""rings"": [[[-13058817, 4026512], [-13058816, ..."
1,Z21,25,None
2,E02,983,"{""rings"": [[[-13068347, 4025106], [-13068347, ..."
3,G12,367,"{""rings"": [[[-13070011, 4019232], [-13070772, ..."
4,17A,288,None
...,...,...,...
213,G17,1,"{""rings"": [[[-13065753, 4017792], [-13065614, ..."
214,I15,0,"{""rings"": [[[-13068909, 4015593], [-13067078, ..."
215,G19,0,"{""rings"": [[[-13062436, 4017578], [-13061289, ..."
216,U02,0,"{""rings"": [[[-13061709, 4025537], [-13060989, ..."


,blockAddress,count,SHAPE
0,3500 BLOCK POLK ST,132,"{'x': -117.466222838411, 'y': 33.904545023372,..."
1,1000 BLOCK SPRUCE ST,188,"{'x': -117.334138494748, 'y': 33.990174826867,..."
2,5th St / Pine St,1,"{'x': -117.384641418932, 'y': 33.982660960342,..."
3,Lila St/jefferson St,1,"{'x': -117.392267626779, 'y': 33.903144477033,..."
4,4500 BLOCK ARLINGTON AVE,35,"{'x': -117.402927090635, 'y': 33.946190570161,..."
...,...,...,...
14109,Stover Ave / Gaylord St,1,"{'x': -117.48100775955, 'y': 33.950900653974, ..."
14110,Manchester Pl / California Ave,1,"{'x': -117.455885144287, 'y': 33.925387404563,..."
14111,6400 BLOCK RHONDA RD,1,"{'x': -117.420405041033, 'y': 33.956169586663,..."
14112,5200 BLOCK MARENGO CT,1,"{'x': -117.500333878109, 'y': 33.9232418253, '..."


# Graph Result


In [10]:
from arcgis.map.popups import FieldInfo, PopupExpressionInfo
import ipywidgets as widgets
from arcgis.features import FeatureCollection



def create_map():
    # Create the map using our featureCollection
    my_map = Map()
    my_map.basemap.basemap =  "arcgis-streets"
        # Update starting field of view, and display the map
    my_map.extent = {'spatialReference': {'wkid': 102100},
                     'xmin': -13087454.153546248, 'ymin': 4009445.945260928,
                     'xmax': -13044725.85473489, 'ymax': 4032377.0537464498}
    return my_map
    

    
def add_layer(my_map, df):
    # setup map data
    fset = FeatureSet.from_dataframe(df)
    feature_collection = FeatureCollection.from_featureset(fset)
    fset2 = FeatureSet.from_dataframe(df2)
    feature_collection2 = FeatureCollection.from_featureset(fset2)

    # setup map popup info
    expression_infos = [
        PopupExpressionInfo(
            title="Name",
            expression="return $feature.NAME;"
        )
    ]
    
    field_infos = [
        FieldInfo(
            fieldName="NAME",
            label="Reporting District",
            visible=True
        ),
        FieldInfo(
            fieldName="Count",
            label="Number of Police Reports",
            visible=True
        )
    ]

    # Add new layer and update popup info
    my_map.content.remove_all()
    my_map.content.add(feature_collection)
    my_map.content.popup(0).edit(expression_infos=expression_infos, field_infos=field_infos)

    # Use smartmapping to make a class_Breaks_renderer with 30 classes and color based on the count field
    my_map.content.renderer(0).smart_mapping().class_breaks_renderer(break_type="color", field="count", num_classes=30)
    
    
    my_map.content.add(feature_collection2)
    
    renderer = my_map.content.renderer(1).renderer
    renderer.symbol.size = 2
    my_map.content.renderer(1).renderer = renderer



def create_combobox(my_map, crime_df, shape_df):
    # Create text/dropdown box
    combobox = widgets.Combobox(
        placeholder="Type or Select Crime Type",
        options=crimeTypes,
        description="Crime Type",
    )
    # Submit on enter/tab clicked only
    combobox.continuous_update = False
    
    #Create a place for output
    output = widgets.Output()
    
    # Handle finishing
    def on_combobox_submit(change):
        with output:
            output.clear_output()
            my_map.content.remove_all()
            print("working")

            queried_df_blockAddress, queried_df_rd, query_string = query_reports(crime_df, combobox.value)
            df = merge_dataframes(queried_df_rd, rd_geometry, "rd")
            add_layer(my_map, df)

            output.clear_output()
            print(f"Displaying: {query_string}")

    
    # Attach the handler
    combobox.observe(on_combobox_submit, names="value")
    
    # Display widgets
    return combobox, output
    

def create_pointsbox(my_map):
    checkbox = widgets.Checkbox(
    value=True,
    description='Display Points'
    )

    def on_checkbox_submit(change):
        if change['new']:
            my_map.content.layer_visibility(1).visibility= True
        else:
            my_map.content.layer_visibility(1).visibility= False
        my_map.content.update_layer(1)
        
        
    checkbox.observe(on_checkbox_submit, names="value")
    return checkbox

def create_opacitybox(my_map):
    checkbox = widgets.Checkbox(
    value=False, 
    description='is_opaque'
    )

    def on_checkbox_submit(change):
        if change['new']:
            renderer = my_map.content.renderer(0).renderer
            for cls in renderer.class_break_infos:
                cls.symbol.color[3] = 100  
            my_map.content.renderer(0).renderer = renderer
        else:
            renderer = my_map.content.renderer(0).renderer
            for cls in renderer.class_break_infos:
                cls.symbol.color[3] = 255  
            my_map.content.renderer(0).renderer = renderer

        
        
    checkbox.observe(on_checkbox_submit, names="value")
    return checkbox

In [11]:
my_map = create_map()
add_layer(my_map, df)
combobox, output = create_combobox(my_map, crime_df, rd_geometry)
pointsbox = create_pointsbox(my_map)
opacitybox = create_opacitybox(my_map)
widgets.VBox([widgets.HBox([combobox, output, pointsbox, opacitybox]), my_map])


In [12]:
renderer = my_map.content.renderer(0).renderer
for cls in renderer.class_break_infos:
    cls.symbol.color[3] = 100  
my_map.content.renderer(0).renderer = renderer

AttributeError: 'SimpleRenderer' object has no attribute 'class_break_infos'

In [15]:
# for val in my_map.content.renderer(0).renderer.class_break_infos:
#     print(val.symbol.outline.color)

print(renderer)

authoring_info=AuthoringInfo(classification_method='esriClassifyEqualInterval', color_ramp=None, fade_ratio=0, field1=None, field2=None, fields=None, flow_theme=None, focus=None, is_auto_generated=None, max_slider_value=None, min_slider_value=None, num_classes=None, standard_deviation_interval=None, statistics=None, type='classedColor', univariate_symbol_style=None, univariate_theme=None, visual_variables=None) background_fill_symbol=None class_break_infos=[ClassBreakInfo(alternate_symbols=None, class_max_value=163.233333, class_min_value=None, description=None, label='0 – 163', symbol=SimpleFillSymbolEsriSFS(color=[247, 251, 255, 100], outline=SimpleLineSymbolEsriSLS(color=[153, 153, 153, 64], marker=None, style='esriSLSSolid', type='esriSLS', width=0.375), style='esriSFSSolid', type='esriSFS')), ClassBreakInfo(alternate_symbols=None, class_max_value=326.466667, class_min_value=None, description=None, label='> 163 – 326', symbol=SimpleFillSymbolEsriSFS(color=[222, 235, 247, 100], outl

In [41]:
my_map.content.renderer(0).renderer

ClassBreaksRenderer(authoring_info=AuthoringInfo(classification_method='esriClassifyEqualInterval', color_ramp=None, fade_ratio=0.0, field1=None, field2=None, fields=None, flow_theme=None, focus=None, is_auto_generated=None, max_slider_value=None, min_slider_value=None, num_classes=None, standard_deviation_interval=None, statistics=None, type='classedColor', univariate_symbol_style=None, univariate_theme=None, visual_variables=None), background_fill_symbol=None, class_break_infos=[ClassBreakInfo(alternate_symbols=None, class_max_value=163.233333, class_min_value=None, description=None, label='0 – 163', symbol=SimpleFillSymbolEsriSFS(color=[247, 251, 255, 100], outline=SimpleLineSymbolEsriSLS(color=[153, 153, 153, 100], marker=None, style='esriSLSSolid', type='esriSLS', width=0.375), style='esriSFSSolid', type='esriSFS')), ClassBreakInfo(alternate_symbols=None, class_max_value=326.466667, class_min_value=None, description=None, label='> 163 – 326', symbol=SimpleFillSymbolEsriSFS(color=[

In [35]:
my_map.content.update_layer(1)